# Import of Librarys

In [ ]:
# Basic
import pandas as pd

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

# Plot and Graphs
import matplotlib.pyplot as plt
import seaborn as sns

import json

# Load Data

In [ ]:
df_original = pd.read_csv('../input/predict-test-scores-of-students/test_scores.csv')
df_original.head()

# Preparação dos Dados

Copiar o modelo inicial para não perde-lo.

In [ ]:
df_adjusted = df_original.copy()

Dividir a base de dados, disponibilizando uma base para treinamento do modelo estatístico e outra base de dados para conferência do modelo treinado.

In [ ]:
df_train, df_test = train_test_split(df_adjusted, test_size=0.2)

df_train1 = df_train.copy()
df_test1 = df_test.copy()

Normalizar as colunas não numéricas que serão relevantes no processo de identificação de padrões. Para isso será utilizado o valor médio do posttest por cada uma das categorias (Valores distintos das colunas de texto). Por questões de teste, usaremos somente os dados do DF de treino e usaremos esta mesma métrica para o DF de teste (o modelo só se baseia em dados de treino).

In [ ]:
textColumns = ['school','school_setting','school_type','classroom','teaching_method','gender','lunch']
mapping = {}

for col in textColumns:
  distinctValues = json.loads(df_train1.groupby(col)['posttest'].mean().to_json())

  df_train1[col] = df_train1[col].map(distinctValues)
  df_test1[col] = df_test1[col].map(distinctValues)
  mapping[col]=distinctValues

print(mapping)

In [ ]:
df_train1.head()

In [ ]:
sns.barplot(x="school", y="posttest", data=df_train1)

# Regressão Linear

## Treinamento
Separar o DF de treino, nos DF de dados e resultado (Aproveitar e remover a coluna de id do aluno, pois não é mais necessária).

In [ ]:
dfX = df_train1[['school','school_setting','school_type','classroom','teaching_method','n_student','gender','lunch','pretest']]
dfy = df_train1.loc[:,'posttest':'posttest'].copy()

dfX.head()

Iniciar o treino do modelo usando regressão linear.

In [ ]:
regr = linear_model.LinearRegression()
regr.fit(dfX, dfy)

## Teste do modelo
Separar o DF de teste, nos DF de dados e resultado.

In [ ]:
dfK = df_test1[['school','school_setting','school_type','classroom','teaching_method','n_student','gender','lunch','pretest']]
dfl = df_test1.loc[:,'posttest':'posttest'].copy()

print("Accurace: %.2f"%(regr.score(dfK,dfl)*100))
print("Coefficients: ")
regr.coef_

Exibir Valores que foram preditos.

In [ ]:
predicted_values = regr.predict(dfK)

dfp = pd.DataFrame(data=predicted_values, index=dfl.index, columns=['predicted'])

mydf = pd.concat([dfl, dfp], axis=1).sort_values(by=['posttest'])

plt.plot(mydf['posttest'], mydf['predicted'], label = 'Predicted');
plt.plot(mydf['posttest'], mydf['posttest'], label = 'Reference');
plt.xlabel('Score')
plt.ylabel('Score')
plt.title('Visual error of predict data')
plt.legend()